<a href="https://colab.research.google.com/github/chandanareddy-enugala/NLP-SLU/blob/main/Assignment_05_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import os
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU
import matplotlib.pyplot as plt
import seaborn as sns

from keras.utils import to_categorical
from numpy import argmax


import re
import os
import sys
import numpy as np
import time
import math
from datetime import datetime

### **Defining Dictionary Functions Class**

In [ ]:
class DictionaryFunctions:
  def __init__(self) -> None:
    pass
  def addVal_to_dictKey(self, D, k, val=1): # Inputs: Dictionary, Key, Value; Ouput: Dictionary (Updated)
    if k in D:
      if type(val)==int:
        D[k] += val
      elif (type(val)==str):
        D[k].append(val)
    else:
      if type(val)==int:
        D[k] = val
      elif (type(val)==str):
        D[k] = [val]
    return D

  def find_probs_of_dict(self, D):
    total = 0
    for key in D.keys():
      total += D[key]
    for key in D.keys():
      D[key] /= total
    return D 

  def get_dict_from_list(self, L):
    D = {}
    for char in L:
      if char in D:
        D[char] += 1
      else:
        D[char] = 1
    return D

  def max_of_dict(self, D):  
    maxProb = 0
    maxKey = ''
    for key, probVal in D.items():
      if probVal>maxProb:
        maxProb = probVal
        maxKey = key
    return (maxKey, maxProb)

### **Loading Dataset**

In [ ]:
def load_data(filePath):
  data_cwe = open(filePath, 'r').read().lower()
  return data_cwe

### **Preprocessing Model Preparation**

In [ ]:
class Language_Translation_Model:
  def __init__(self, lang="") -> None:
    self.lang = lang
    self.data = pd.DataFrame(columns=['sentence'])
    self.all_Unique_Characters = None
    self.length_all_Unique_Characters = None
    self.length_max_sentence = None
    self.sentenceArray_to_OHE = None
    self.char_to_num = None
    self.num_to_char = None
    # self.ASCII_Chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'
  
  def sentence_to_array(self, sentence):
    sentenceArray = [self.char_to_num[char] for char in sentence]
    #padding = [0 for _ in range(self.length_all_Unique_Characters-len(sentenceArray))]
    #sentenceArray = np.array(sentenceArray+padding)
    return sentenceArray
  
  def sentenceArray_to_OHE1(self, sentenceArray):
    sentenceArray_OHE = to_categorical(sentenceArray)
    (req_rows, req_cols) = (self.length_max_sentence, self.length_all_Unique_Characters)
    (actual_rows, actual_cols) = sentenceArray_OHE.shape
    sentenceArray_OHE = np.concatenate((sentenceArray_OHE, np.zeros((req_rows-actual_rows, actual_cols), dtype='float32')), axis=0)
    sentenceArray_OHE = np.concatenate((sentenceArray_OHE, np.zeros((req_rows, req_cols-actual_cols), dtype='float32')), axis=1)
    return sentenceArray_OHE

  def OHE_to_sentenceArray(self, sentence_array_OHE):
    sentenceArray = []
    for OHE in sentence_array_OHE:
      if sum(OHE)!=0:
        sentenceArray.append(argmax(OHE))
    return sentenceArray
  
  def preprocess_data(self, text):
    # Cleaning --------------------------
    text = text.replace("\n", " ") 
    for char in [' .', ' ,', ' ;', ' ?', ' !', '( ', ' )']:
      text = text.replace(char, char.strip())
    text = text.replace("<s>", "")
    
    # =========================== CHARACTERS ===========================
    # Unique Characters -----------------
    self.all_Unique_Characters = list(set(text.replace("</s>", "")))
    self.all_Unique_Characters.sort()
    
    # Max Length of Unique Characters ---
    self.length_all_Unique_Characters = len(self.all_Unique_Characters)
    
    # Preparing Character to Number and vice-versa -----
    self.char_to_num = {char:num for num, char in enumerate(self.all_Unique_Characters)}
    self.num_to_char = {val:key for key, val in self.char_to_num.items()}

    # =========================== SENTENCES ===========================
    # Sentences -------------------------
    sentences = text.split("</s>")[:-1]
    sentences = [sentence.strip() for sentence in sentences]
    
    # Max Length of Sentences -----------
    self.length_max_sentence = max([len(sentence) for sentence in sentences])
    
    # Preparing OHE Array Variable ------
    self.sentenceArray_to_OHE = np.zeros((self.length_max_sentence, self.length_all_Unique_Characters), dtype='float32')
    
    # =========================== DATAFRAME ===========================
    # Storing sentences in dataframe ----
    self.data['sentence'] = sentences
    # Converting Sentence to Array Numbers ----
    self.data['sentence_array'] = self.data['sentence'].apply(lambda x: self.sentence_to_array(x))
    # Converting Array Numbers to One-Hot-Encoding (OHE) ----
    self.data['sentence_array_OHE'] = self.data['sentence_array'].apply(lambda x: self.sentenceArray_to_OHE1(x))

    return self.data

In [ ]:
filePath = "/content/drive/MyDrive/NLP/NLP_Assingnmnet05/data/train-05/train-source.txt"
train_input_data = load_data(filePath)

In [ ]:
Input_LTM = Language_Translation_Model()
Input_data = Input_LTM.preprocess_data(train_input_data[0:10000])
Input_data.head()

,sentence,sentence_array,sentence_array_OHE
0,"cinnte go leór, thiocfadh dóbhtha bás a fhaghá...","[16, 22, 26, 26, 31, 18, 0, 20, 27, 0, 24, 18,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,(bhí sé follasach go rabh an poll sin ag foscl...,"[4, 15, 21, 36, 0, 30, 35, 0, 19, 27, 24, 24, ...","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
2,) d'fhéadfadh siad bás ' fhagháil ar a bhruach...,"[5, 0, 17, 3, 19, 21, 35, 14, 17, 19, 14, 17, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,..."
3,thiocfadh dóbhtha fosta lámh a chur ina mbás f...,"[31, 21, 22, 27, 16, 19, 14, 17, 21, 0, 17, 37...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,"' na dhiaidh sin, bhí rud éigin do-chreidte ag...","[3, 0, 26, 14, 0, 17, 21, 22, 14, 22, 17, 21, ...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
filePath = "/content/drive/MyDrive/NLP/NLP_Assingnmnet05/data/train-05/train-target.txt"
train_target_data = load_data(filePath)

In [ ]:
Output_LTM = Language_Translation_Model()
Output_data = Output_LTM.preprocess_data(train_target_data[0:10000])
Output_data.head()

,sentence,sentence_array,sentence_array_OHE
0,"cinnte go leor, thiocfadh dóibh bás a fháil ar...","[15, 21, 25, 25, 30, 17, 0, 19, 26, 0, 23, 17,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,bhí sé follasach go raibh an poll sin ag foscl...,"[14, 20, 35, 0, 29, 34, 0, 18, 26, 23, 23, 13,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,d'fhéadfadh siad bás a fháil ar a bhruach agus...,"[16, 2, 18, 20, 34, 13, 16, 18, 13, 16, 20, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,thiocfadh dóibh fosta lámh a chur ina mbás féi...,"[30, 20, 21, 26, 15, 18, 13, 16, 20, 0, 16, 36...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,ina dhiaidh sin bhí rud éigin dochreidte agus ...,"[21, 25, 13, 0, 16, 20, 21, 13, 21, 16, 20, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
encoder_input_data = np.array([record for record in Input_data['sentence_array_OHE']])
decoder_input_data = np.array([np.concatenate((np.zeros((record.shape[0], 1), dtype='float32'), record[:, 1:]), axis=1) for record in Output_data['sentence_array_OHE']])
decoder_target_data = np.array([record for record in Output_data['sentence_array_OHE']])

### **Model Preparation**

In [ ]:
batch_size = 128
epochs=1
latent_dim=256
num_samples=10000

In [ ]:
encoder_inputs = Input(shape=(None, Input_LTM.length_all_Unique_Characters))
encoder = LSTM(latent_dim, return_state=True) # latent_dim = 256
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
#Decoder
decoder_inputs = Input(shape=(None, Output_LTM.length_all_Unique_Characters))

# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(Output_LTM.length_all_Unique_Characters, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Defining Model ------------------
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Training Model ------------------
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

1/1 [==============================] - 8s 8s/step - loss: 1.2344 - accuracy: 0.3334 - val_loss: 1.2413 - val_accuracy: 0.7151


**Predictions**

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_states_inputs = [Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
test_input_seq = data.loc[0, 'sentence']
print(test_input_seq)
encoder_model.predict(test_input_seq)

cinnte go leór, thiocfadh dóbhtha bás a fhagháil ar imeall an phuill udaí.


IndexError: ignored

**Save Model**

In [ ]:
model.save('s2s.h5')

## **Rough Work**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
#enc.n_values_
# array([2, 3, 4])
enc.feature_indices_
# array([0, 2, 5, 9], dtype=int32)
enc.transform([[0, 1, 1]]).toarray()
# array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.]])

AttributeError: ignored

In [ ]:
allUniqueCharacters = []
for sentence in input_texts:
  allUniqueCharacters += list(sentence)
allUniqueCharacters = list(set(allUniqueCharacters))
len(allUniqueCharacters)

71

In [ ]:
char_to_num = {char:num for num, char in enumerate(allUniqueCharacters, start=1)}

In [ ]:
max_sent_length = max([len(sent) for sent in  input_texts])

In [ ]:
def sentenceToArray(x):
  array = [char_to_num[char] for char in x]
  array += [0 for _ in range(max_sent_length - len(array))]
  return array

In [ ]:
LTM1 = Language_Translation_Model()
input_text_array = pd.Series(input_texts).apply(lambda x: sentenceToArray(x))
input_text_array

0       [7, 68, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
1       [27, 22, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [27, 22, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [50, 3, 6, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
4       [50, 3, 6, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
                              ...                        
9995    [71, 70, 10, 12, 8, 68, 3, 12, 22, 6, 15, 3, 7...
9996    [71, 70, 10, 12, 8, 68, 3, 12, 22, 6, 15, 3, 7...
9997    [71, 70, 10, 12, 8, 68, 3, 12, 22, 6, 15, 3, 7...
9998    [71, 70, 10, 12, 8, 68, 3, 12, 22, 6, 15, 3, 7...
9999    [71, 70, 10, 12, 8, 68, 3, 12, 22, 6, 15, 3, 7...
Length: 10000, dtype: object

In [ ]:
def sentenceArray_to_OHE1(sentenceArray):
  sentenceArray_OHE = to_categorical(sentenceArray)
  return sentenceArray_OHE

In [ ]:
input_text_OHE = input_text_array.apply(lambda x: sentenceArray_to_OHE1(x))

In [ ]:
input_text_OHE[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
#input_text_OHE[999]
L0 = [argmax(char) for char in input_text_OHE[999]]
print(L0)
print([num_to_char0[num] for num in L0 if num!=0])

[66, 10, 12, 29, 39, 22, 61, 10, 33, 16, 0, 0, 0, 0, 0, 0]
['W', 'e', ' ', 'w', 'a', 'i', 't', 'e', 'd', '.']


In [ ]:
num_to_char0 = {num:char for char, num in char_to_num.items()}
num_to_char0[66]

'W'

In [ ]:
argmax(encoder_input_data[999][0])

42

In [ ]:
encoder_input_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
#input_text_OHE[999]
L1 = [argmax(char) for char in encoder_input_data[999]]
print(L1)
print([num_to_char1[num] for num in L1 if num!=0])

[42, 48, 0, 66, 44, 52, 63, 48, 47, 8, 0, 0, 0, 0, 0, 0]
['W', 'e', 'w', 'a', 'i', 't', 'e', 'd', '.']


In [ ]:
num_to_char1[42]

'W'

In [ ]:
num_to_char1 = {num:char for char, num in input_token_index.items()}

In [ ]:
max(input_text_array[9999])
#input_text_OHE[999]
L0 = [argmax(char) for char in input_text_OHE[999]]
print(L0)
print([num_to_char0[num] for num in L0 if num!=0])

71

In [ ]:
filePath = "/content/drive/MyDrive/NLP/NLP_Assingnmnet05/data/train-05/fra.txt"
with open(filePath, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [ ]:
num_samples=10000
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples:', len(input_texts))

print('Number of unique input tokens:', num_encoder_tokens)

print('Number of unique output tokens:', num_decoder_tokens)

print('Max sequence length for inputs:', max_encoder_seq_length)

print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            '''
            decoder_target_data will be ahead by one timestep
            and will not include the start character.
            
            '''
            
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
for t, char in enumerate(target_texts[1000]):
  print(t, char)

0 	
1 N
2 o
3 u
4 s
5  
6 s
7 o
8 m
9 m
10 e
11 s
12  
13 a
14 l
15 l
16 é
17 s
18  
19 à
20  
21 p
22 i
23 e
24 d
25 .
26 



In [ ]:
input_texts[1000], target_texts[1000]

('We walked.', '\tNous sommes allés à pied.\n')

In [ ]:
encoder_input_data[1000].shape

(16, 71)

In [ ]:
print([argmax(row) for row in encoder_input_data[1000]])

[42, 48, 0, 66, 44, 55, 54, 48, 47, 8, 0, 0, 0, 0, 0, 0]


In [ ]:
decoder_input_data[1000].shape

(59, 92)

In [ ]:
print([argmax(row) for row in decoder_input_data[2000]])

[0, 26, 44, 52, 63, 48, 62, 10, 55, 48, 2, 46, 58, 56, 56, 48, 2, 52, 55, 2, 49, 44, 64, 63, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [ ]:
decoder_target_data[1000].shape

(59, 92)

In [ ]:
print([argmax(row) for row in decoder_target_data[2000]])

[26, 44, 52, 63, 48, 62, 10, 55, 48, 2, 46, 58, 56, 56, 48, 2, 52, 55, 2, 49, 44, 64, 63, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [ ]:
encoder_input_data[0][:, 1:].shape

(16, 70)

In [ ]:
" ljflaj ".strip()

'ljflaj'

In [ ]:
n=15
LM_15 = Language_Model(n, 'swe')

In [ ]:
train_input_data_sentences = LM_15.get_Sentences_from_Text(train_input_data)

In [ ]:
print("".join(train_input_data_sentences[0]))

<s>
cinnte
go
leór
,
thiocfadh
dóbhtha
bás
a
fhagháil
ar
imeall
an
phuill
udaí
.


In [ ]:
train_input_data[0:80]

'<s>\ncinnte\ngo\nleór\n,\nthiocfadh\ndóbhtha\nbás\na\nfhagháil\nar\nimeall\nan\nphuill\nudaí\n.'

### **Train the Language Model : SWE**

**5-gram Model**

In [ ]:
startTime = datetime.now()

LM_5 = Language_Model(5)
LM_5.fit(train_data)

endTime = datetime.now()
print("\n")
print(f"Code running time : {endTime-startTime}")

==== TRAINING IS COMPLETED ====
Success: ngrams & ngrams_nextChars are stored successfully


Code running time : 0:00:01.562456


In [ ]:
log2loss = LM_5.evaluate(train_data)
print(log2loss)

1.209649119639972


In [ ]:
log2loss = LM_5.evaluate(test_data)
print(log2loss)

1.1544019103445355


In [ ]:
n = 5

startTime = datetime.now()

LM_5 = Language_Model(n, 'swe')
LM_5.fit(train_data)

print("\n")
print(f"Training Log2Loss : {LM_5.evaluate(train_data)}") 
print(f"Testing Log2Loss : {LM_5.evaluate(test_data)}") 

endTime = datetime.now()
print("\n")
print(f"Code running time : {endTime-startTime}")

==== TRAINING IS COMPLETED ====
Success: ngrams & ngrams_nextChars are stored successfully


Training Log2Loss : 1.2688606397085775
Testing Log2Loss : 1.206416137765356


Code running time : 0:00:04.303345


**10-gram Model**

In [ ]:
n = 10

startTime = datetime.now()

LM_10 = Language_Model(n, 'swe')
LM_10.fit(train_data)

print("\n")
print(f"Training Log2Loss : {LM_10.evaluate(train_data)}") 
print(f"Testing Log2Loss : {LM_10.evaluate(test_data)}") 

endTime = datetime.now()
print("\n")
print(f"Code running time : {endTime-startTime}")

==== TRAINING IS COMPLETED ====
Success: ngrams & ngrams_nextChars are stored successfully
Training Log2Loss : 0.3780673061919561
Testing Log2Loss : 0.27591094078507133


Code running time : 0:00:03.806859


**15-gram Model**

In [ ]:
n = 15

startTime = datetime.now()

LM_15 = Language_Model(n, 'swe')
LM_15.fit(train_data)

print("\n")
#print(f"Training Log2Loss : {LM_15.evaluate(train_data)}") 
print(f"Testing Log2Loss : {LM_15.evaluate(test_data)}") 

endTime = datetime.now()
print("\n")
print(f"Code running time : {endTime-startTime}")

==== TRAINING IS COMPLETED ====
Success: ngrams & ngrams_nextChars are stored successfully


Testing Log2Loss : 0.06150412268756698


Code running time : 0:00:05.202777


**Testing**

In [ ]:
LM_15 = Language_Model(n, 'swe')
LM_15.fit(train_data)

==== TRAINING IS COMPLETED ====
Success: ngrams & ngrams_nextChars are stored successfully


In [ ]:
print(f"Training Log2Loss : {LM_15.evaluate(train_data)}") 

print("_ngrams_Available : ", len(LM_15._ngrams_Available))
print("_ngrams_nextChars_Available : ", len(LM_15._ngrams_nextChars_Available))

print("_ngrams_notAvailable : ", len(LM_15._ngrams_notAvailable))
print("_ngrams_nextChars_notAvailable : ", len(LM_15._ngrams_nextChars_notAvailable))

Training Log2Loss : 0.11459748277321753
_ngrams_Available :  603432
_ngrams_nextChars_Available :  603431
_ngrams_notAvailable :  0
_ngrams_nextChars_notAvailable :  1


In [ ]:
print(f"Testing Log2Loss : {LM_15.evaluate(test_data)}") 

print("_ngrams_Available : ", len(LM_15._ngrams_Available))
print("_ngrams_nextChars_Available : ", len(LM_15._ngrams_nextChars_Available))

print("_ngrams_notAvailable : ", len(LM_15._ngrams_notAvailable))
print("_ngrams_nextChars_notAvailable : ", len(LM_15._ngrams_nextChars_notAvailable))

Testing Log2Loss : 0.06150412268756698
_ngrams_Available :  15920
_ngrams_nextChars_Available :  13311
_ngrams_notAvailable :  45797
_ngrams_nextChars_notAvailable :  48406


In [ ]:
len(test_data)

61717

**Testing 2**

In [ ]:
n=15
LM_15 = Language_Model(n, 'swe')
LM_15.fit(train_data)

TypeError: ignored

In [ ]:
print(f"Training Log2Loss : {LM_15.evaluate(train_data)}") 

print("_ngrams_Available : ", len(LM_15._ngrams_Available))
print("_ngrams_nextChars_Available : ", len(LM_15._ngrams_nextChars_Available))

print("_ngrams_notAvailable : ", len(LM_15._ngrams_notAvailable))
print("_ngrams_nextChars_notAvailable : ", len(LM_15._ngrams_nextChars_notAvailable))

Training Log2Loss : 11.443842603736776
_ngrams_Available :  603432
_ngrams_nextChars_Available :  603431
_ngrams_notAvailable :  0
_ngrams_nextChars_notAvailable :  1


In [ ]:
print(f"Testing Log2Loss : {LM_15.evaluate(test_data)}") 

print("_ngrams_Available : ", len(LM_15._ngrams_Available))
print("_ngrams_nextChars_Available : ", len(LM_15._ngrams_nextChars_Available))

print("_ngrams_notAvailable : ", len(LM_15._ngrams_notAvailable))
print("_ngrams_nextChars_notAvailable : ", len(LM_15._ngrams_nextChars_notAvailable))

Testing Log2Loss : 11.48324833516333
_ngrams_Available :  61669
_ngrams_nextChars_Available :  61345
_ngrams_notAvailable :  48
_ngrams_nextChars_notAvailable :  372


**Rough**

In [ ]:
train_data_Char_List = list(train_data)

DF = DictionaryFunctions()
train_data_Char_Dict = DF.get_dict_from_list(train_data_Char_List)
train_data_Char_Dict

{'c': 5446,
 'h': 20755,
 'i': 46266,
 'k': 23425,
 'a': 87267,
 'l': 33255,
 'e': 32796,
 ' ': 84914,
 'v': 5778,
 'n': 38966,
 'o': 26873,
 'g': 16766,
 'f': 2446,
 'y': 17481,
 'w': 31352,
 'u': 40148,
 'b': 7463,
 "'": 2502,
 'm': 25905,
 's': 10737,
 ',': 7106,
 'j': 1545,
 'd': 9800,
 't': 7034,
 '.': 5269,
 'p': 1141,
 'z': 8054,
 '"': 2167,
 '!': 341,
 '?': 421,
 '-': 12,
 '\n': 1}

In [ ]:
DF = DictionaryFunctions()
train_data_Char_Prob = DF.find_probs_of_dict(train_data_Char_Dict)
train_data_Char_Prob

{'c': 0.009025043418313911,
 'h': 0.03439492768033515,
 'i': 0.07667143936682178,
 'k': 0.0388196184491376,
 'a': 0.14461778626257804,
 'l': 0.05510977210356759,
 'e': 0.0543491230163465,
 ' ': 0.14071842394834877,
 'v': 0.009575229686194965,
 'n': 0.06457397022365403,
 'o': 0.04453360113484204,
 'g': 0.027784406527993213,
 'f': 0.004053480756738124,
 'y': 0.028969295629002106,
 'w': 0.051956144188574686,
 'u': 0.0665327659123149,
 'b': 0.012367590714446697,
 "'": 0.004146283259754206,
 'm': 0.042929443582706915,
 's': 0.01779322276577974,
 ',': 0.011775974757719179,
 'j': 0.00256035477071153,
 'd': 0.016240438027814236,
 't': 0.01165665725384136,
 '.': 0.00873172122128094,
 'p': 0.0018908509989526575,
 'z': 0.013346988558777128,
 '"': 0.0035911254292115766,
 '!': 0.0005651009558657811,
 '?': 0.0006976759601744687,
 '-': 1.9886250646303147e-05,
 '\n': 1.6571875538585954e-06}

In [ ]:
sum = 0
for key in train_data_Char_Prob:
  sum += train_data_Char_Prob[key]
print(sum)

1.0000000000000002


In [ ]:
test_data_Char_List = list(test_data)

DF = DictionaryFunctions()
test_data_Char_Dict = DF.get_dict_from_list(test_data_Char_List)
test_data_Char_Prob = DF.find_probs_of_dict(test_data_Char_Dict)

sum = 0
for key in test_data_Char_Prob:
  sum += test_data_Char_Prob[key]
print(sum)

0.9999999999999999


In [ ]:
print(len(train_data))

603432


In [ ]:
ASCII_Chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'
all_characters_in_data = re.findall(r"[%s]"%ASCII_Chars, train_data)
print(len(all_characters_in_data))

603431
